## Importing Dependencies

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils


Using TensorFlow backend.


## Loading of Data

In [16]:
text = (open("demofile.txt").read())
text=text.lower()
# print(text)

## Creating character/word mappings

In [4]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}
# print(n_to_char)
# print(char_to_n)

{0: '\n', 1: '\r', 2: ' ', 3: '!', 4: '"', 5: '#', 6: '$', 7: '%', 8: "'", 9: '(', 10: ')', 11: '*', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '?', 29: '@', 30: '[', 31: ']', 32: 'a', 33: 'b', 34: 'c', 35: 'd', 36: 'e', 37: 'f', 38: 'g', 39: 'h', 40: 'i', 41: 'j', 42: 'k', 43: 'l', 44: 'm', 45: 'n', 46: 'o', 47: 'p', 48: 'q', 49: 'r', 50: 's', 51: 't', 52: 'u', 53: 'v', 54: 'w', 55: 'x', 56: 'y', 57: 'z'}
{'\n': 0, '\r': 1, '!': 3, ' ': 2, '#': 5, '"': 4, '%': 7, '$': 6, "'": 8, ')': 10, '(': 9, '*': 11, '-': 13, ',': 12, '/': 15, '.': 14, '1': 17, '0': 16, '3': 19, '2': 18, '5': 21, '4': 20, '7': 23, '6': 22, '9': 25, '8': 24, ';': 27, ':': 26, '?': 28, '@': 29, '[': 30, ']': 31, 'a': 32, 'c': 34, 'b': 33, 'e': 36, 'd': 35, 'g': 38, 'f': 37, 'i': 40, 'h': 39, 'k': 42, 'j': 41, 'm': 44, 'l': 43, 'o': 46, 'n': 45, 'q': 48, 'p': 47, 's': 50, 'r': 49, 'u': 52, 't': 51, 'w': 54, 'v': 5

## Data pre-processing

In [15]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in  (0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
#     print("label is " ,label)
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
#     print(sequence)
# print(X[:100])
# # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
# print(Y[:100])

IndexError: string index out of range

In [7]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

## A deeper model

In [8]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)

model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')


Epoch 1/100
 10050/122675 [=>............................] - ETA: 3:20:13 - loss: 3.1102

In [ ]:
model.load_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

## Generating Text

In [ ]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


In [ ]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt